In [24]:
import fitz
import easyocr
from PIL import Image
import io
import numpy as np

In [25]:

reader = easyocr.Reader(['pt'], gpu=True)

def extrair_texto_pdf_sem_poppler(caminho_pdf: str) -> str:
    texto_total = ""

    # Abre o PDF
    doc = fitz.open(caminho_pdf)

    texto_total = ""

    for pagina in doc:
        texto = pagina.get_text()
        if texto.strip():
            texto_total += texto + "\n"

    if texto_total.strip():
        return texto_total.strip()

    for pagina in doc:
        # Renderiza a página em imagem (matriz de pixels)
        pix = pagina.get_pixmap(dpi=200)
        imagem_bytes = pix.tobytes("png")

        # Converte os bytes em imagem PIL
        imagem_pil = Image.open(io.BytesIO(imagem_bytes)).convert("RGB")

        # Converte PIL para array numpy (formato aceito pelo EasyOCR)
        imagem_array = np.array(imagem_pil)

        # Usa OCR na imagem
        resultado = reader.readtext(imagem_array, detail=0)
        texto_total += "\n".join(resultado) + "\n"

    doc.close()
    return texto_total.strip()


Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.


In [26]:
#print(extrair_texto_pdf_sem_poppler("NFSe_00003018_06533027 - Fabio.pdf"))

texto_da_nota = extrair_texto_pdf_sem_poppler("NFSe_00003018_06533027 - Fabio.pdf")

#print(extrair_texto_pdf_sem_poppler("Aposta_250707_125432.pdf"))

print(texto_da_nota)


Número da Nota
PREFEITURA DA CIDADE DO RIO DE JANEIRO
00003018
SECRETARIA MUNICIPAL DE FAZENDA
Data
Hora de Emissão
08/07/2025 12:11:58
NOTA FISCAL DE SERVIÇOS ELETRÔNICA
NFS-e
Código de Verificação
20250708u23112919000172123112919000172
NOTA CARIOCA
PJIS-9JTY
PRESTADOR DE SERVIÇOS
CPFICNPJ:
23.112.919/0001-72
Inscrição Municipal: 0.653.302-7
Inscrição Estadual:
NomelRazão Social:
AVANTE PSICOLOGIA E DESENVOLVIMENTO PROFISSIONAL LTDA
Nome Fantasia:
Tel.
2133460141
Endereço: AVN MERITI 1795, SAL 205
VILA DA PENHA
CEP: 21211-007
Município: RIO DE JANEIRO
UF
RJ
E-mail: legal@conthen.com.br
TOMADOR DE SERVIÇOS
CPFICNPJ:
112.053.917-03
Inscrição Municipal:
Inscrição Estadual:
Nome/Razão Social:
FABIO SOUZA DA SILVA MOTA
Endereço: RUA CABO HERCULANO 245
BRAS DE PINA
CEP
21211-740
Tel.
Município: RIO DE JANEIRO
UF:RJ
E-mail:
DISCRIMINAÇÃO DOs SERVIÇOS
Consulta
emn
Psicologia
07/07/2025
RS
150, 00
14/07/2025
R$
150
00
21/07/2025
R$
150_
00
28/07/2025
R$
150, 00
Psicóloga
res pons ável
Solange


In [27]:
import os
from langchain.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

apiKey = os.environ["open_ai_key"]

def extrair_dados_pdf(path: str) -> str:

    template = """
     Extraia as seguintes informações do texto abaixo e retorne um JSON com as chaves:
    'nome do prestador', 'cpf ou CNPJ do prestador', 'data', 'tipo de serviço', 'valor_total', número da nota.

    Texto da nota:
    """

    prompt_template = PromptTemplate.from_template(template)

    llm = ChatOpenAI(model="gpt-4o",api_key=apiKey)
    texto = texto_da_nota
    prompt = f"""
    Extraia as seguintes informações do texto abaixo e retorne um JSON com as chaves:
    'prestador', 'cnpj', 'data', 'procedimento', 'valor_total'.

    Texto da nota:
    {texto}
    """
    return llm.predict(prompt)

In [ ]:
print(extrair_dados_pdf(""))

C:\Users\F_Mota\AppData\Local\Temp\ipykernel_35460\3473515090.py:27: LangChainDeprecationWarning: The method `BaseChatModel.predict` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  return llm.predict(prompt)


```json
{
    "prestador": "AVANTE PSICOLOGIA E DESENVOLVIMENTO PROFISSIONAL LTDA",
    "cnpj": "23.112.919/0001-72",
    "data": "08/07/2025",
    "procedimento": "Consulta em Psicologia",
    "valor_total": "600,00"
}
```
